In [ ]:
import numpy as np
import pandas as pd
from typing import Dict
from tempfile import gettempdir
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn 
from torch.utils.data import DataLoader
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import l5kit
print(l5kit.__version__)

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer

from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset, write_gt_csv
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace

from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import os
os.environ["L5KIT_DATA_FOLDER"] = "../input/lyft-motion-prediction-autonomous-vehicles"
dm = LocalDataManager(None)

In [ ]:
cfg = {
    'format_version': 4,
    'model_params': {'model_architecture': 'resnet18',
                     'history_num_frames': 10,
                     'history_step_size': 1,
                     'history_delta_time': 0.1,
                     'future_num_frames': 50,
                     'future_step_size': 1,
                     'future_delta_time': 0.1},
    'raster_params': {'raster_size': [224, 224],
                      'pixel_size': [0.5, 0.5],
                      'ego_center': [0.25, 0.5],
                      'map_type': 'py_semantic',
                      'satellite_map_key': 'aerial_map/aerial_map.png',
                      'semantic_map_key': 'semantic_map/semantic_map.pb',
                      'dataset_meta_key': 'meta.json',
                      'filter_agents_threshold': 0.5,
                     'disable_traffic_light_faces':False},
    'example_data_loader': {'key': 'scenes/sample.zarr',
                        'batch_size': 4,
                        'shuffle': False,
                        'num_workers': 0}}

rasterizer = build_rasterizer(cfg, dm)

example_cfg = cfg["example_data_loader"]
example_zarr = ChunkedDataset(dm.require(example_cfg["key"])).open()
dataset1 = AgentDataset(cfg, example_zarr, rasterizer)

## load model

In [ ]:
class LyftMultiModel(nn.Module):

    def __init__(self, cfg: Dict, num_modes=3):
        super().__init__()
        
        """backbone network"""
        self.backbone = models.resnet18()
        self.backbone_out_features = 512  #for resnet18 and resnet34; 2048 for the other resnets
        
        
        self.num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        self.num_in_channels = 3 + self.num_history_channels
        
        self.num_modes = num_modes
        self.num_targets = cfg["model_params"]["future_num_frames"]
        self.num_preds = 2 * self.num_targets * self.num_modes 
        

        self.backbone.conv1 = nn.Conv2d(
            self.num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )

        self.head = nn.Sequential(
            nn.Linear(in_features=self.backbone_out_features, out_features=4096),
            nn.Linear(4096, out_features = self.num_preds + self.num_modes)
        )

        
    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)

        x = self.head(x)

        confidences, preds = torch.split(x, [self.num_modes, self.num_preds], dim=1)
      
        return  confidences, preds.view(-1, self.num_modes , self.num_targets, 2) 
    
model = LyftMultiModel(cfg).to(device)
model.load_state_dict(torch.load('../input/lyft-motion-resnet18-multi-mode-training-5/lyft_multi_mode_model5_cpu.pth'))
model.eval()

## plot predictions

In [ ]:
def plot_target(dataset, idx, COLOR = TARGET_POINTS_COLOR):
    data = dataset[idx]

    # convert raster into rgb
    im = dataset.rasterizer.to_rgb(data["image"].transpose(1, 2, 0)) 
    # transform points from agent local coordinate to world coordinate
    #positions_in_world = transform_points(data["target_positions"], data["world_from_agent"])
    # transform from world coordinates onto raster
    #positions_in_raster = transform_points(positions_in_world, data["raster_from_world"])
    positions_in_raster = transform_points(data["target_positions"], data["raster_from_agent"])
    draw_trajectory(im, positions_in_raster, COLOR, yaws=data["target_yaws"])
    plt.imshow(im[::-1])

In [ ]:
def plot_predictions(dataset, i, model):
    # visualize predictions
    data = dataset[i]
    
    x = torch.FloatTensor(data["image"]).unsqueeze(0).to(device)
    confidences, preds = model(x)
    conf_probs = torch.softmax(confidences, 1)

    conf_probs = conf_probs.squeeze()
    preds = preds.squeeze()

    fig = plt.figure( figsize=(16, 16))

    ax = fig.add_subplot(131)
    idx = 0
    ax.set_title(str(round(conf_probs[idx].item(), 4)))

    im = dataset.rasterizer.to_rgb(data["image"].transpose(1, 2, 0))  # convert raster into rgb
    # transform points from agent local coordinate onto raster
    positions_in_raster = transform_points(preds[idx].cpu().detach().numpy(), data["raster_from_agent"])
    draw_trajectory(im, positions_in_raster, PREDICTED_POINTS_COLOR, yaws=data["target_yaws"])
    ax.imshow(im[::-1])


    ax = fig.add_subplot(132)
    idx = 1
    ax.set_title(str(round(conf_probs[idx].item(), 4)))

    im = dataset.rasterizer.to_rgb(data["image"].transpose(1, 2, 0))  # convert raster into rgb
    # transform points from agent local coordinate onto raster
    positions_in_raster = transform_points(preds[idx].cpu().detach().numpy(), data["raster_from_agent"])
    draw_trajectory(im, positions_in_raster, (255, 0 , 0), yaws=data["target_yaws"])
    ax.imshow(im[::-1])


    ax = fig.add_subplot(133)
    idx = 2

    ax.set_title(str(round(conf_probs[idx].item(), 4)))
    im = dataset.rasterizer.to_rgb(data["image"].transpose(1, 2, 0))  # convert raster into rgb
    # transform points from agent local coordinate onto raster
    positions_in_raster = transform_points(preds[idx].cpu().detach().numpy(), data["raster_from_agent"])
    draw_trajectory(im, positions_in_raster, (255, 255 , 0), yaws=data["target_yaws"])
    ax.imshow(im[::-1])

    plt.show()

In [ ]:
idx = 500
plot_target(dataset1, idx)
plot_predictions(dataset1,idx , model)

In [ ]:
idx = 1000
plot_target(dataset1, idx)
plot_predictions(dataset1,idx , model)

In [ ]:
idx = 2500
plot_target(dataset1, idx)
plot_predictions(dataset1,idx , model)

## animation

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML
from IPython.display import display, clear_output

import PIL

rc('animation', html='jshtml')

def animate_solution(images):

    def animate(i):
        im.set_data(images[i])
 
    fig, ax = plt.subplots()
    im = ax.imshow(images[0])
    
    return animation.FuncAnimation(fig, animate, frames=len(images), interval=60)

ego_dataset = EgoDataset(cfg, example_zarr, rasterizer)

sence 8, 14

In [ ]:
scene_idx = 25
indexes = ego_dataset.get_scene_indices(scene_idx)
print(len(indexes))
plot_target(ego_dataset, indexes[0])

In [ ]:
def animate_prediction(images, confs):

    def animate(i):
        im0.set_data(images[i][0])
        im1.set_data(images[i][1])
        im2.set_data(images[i][2])
        im3.set_data(images[i][3])
        axes[0].set_title(str(round(confs[i,0], 2)))
        axes[1].set_title(str(round(confs[i,1], 2)))
        axes[2].set_title(str(round(confs[i,2], 2)))
        axes[3].set_title('ground truth')
 
    fig, axes = plt.subplots(1,4, figsize=(16,4))

    im0 = axes[0].imshow(images[0][0])
    im1 = axes[1].imshow(images[0][1])
    im2 = axes[2].imshow(images[0][2])
    im3 = axes[3].imshow(images[0][3])
    
    return animation.FuncAnimation(fig, animate, frames=len(images), interval=60*4)

In [ ]:
COLORS = [PREDICTED_POINTS_COLOR, (255, 0 , 0), (255, 255 , 0)]

def make_anim(scene_idx):
    indexes = ego_dataset.get_scene_indices(scene_idx)
    images, confs = [], []

    for idx in indexes:
        ims = []
        data = ego_dataset[idx]
    
        x = torch.FloatTensor(data["image"]).unsqueeze(0).to(device)
        confidences, preds = model(x)
        conf_probs = torch.softmax(confidences, 1)
        preds = preds.squeeze()
    
        for idx in range(3):
            im = ego_dataset.rasterizer.to_rgb(data["image"].transpose(1, 2, 0)) 
            positions_in_raster = transform_points(preds[idx].cpu().detach().numpy(), data["raster_from_agent"])
            draw_trajectory(im, positions_in_raster, COLORS[idx], yaws=data["target_yaws"])
            ims.append(PIL.Image.fromarray(im[::-1]))
    
        # target
        im = ego_dataset.rasterizer.to_rgb(data["image"].transpose(1, 2, 0)) 
        positions_in_raster = transform_points(data["target_positions"], data["raster_from_agent"])
        draw_trajectory(im, positions_in_raster, TARGET_POINTS_COLOR, yaws=data["target_yaws"])
        ims.append(PIL.Image.fromarray(im[::-1]))
    
        confs.append(conf_probs.detach().numpy())
        images.append(ims)
    
    confs = np.concatenate(confs)
    return animate_prediction(images, confs)

In [ ]:
anim = make_anim(8)
HTML(anim.to_jshtml())

In [ ]:
anim = make_anim(14)
HTML(anim.to_jshtml())

In [ ]:
anim = make_anim(25)
HTML(anim.to_jshtml())